# ETL Pipeline: PDF to PostgreSQL pgvector

이 노트북은 PDF 파일을 읽어서 마크다운으로 변환하고, 청크로 나눈 후 임베딩하여 PostgreSQL pgvector에 적재하는 전체 ETL 파이프라인을 구현합니다.

## Pipeline Overview
1. **Chapter-based Markdown Conversion**: 챕터 배열을 받아서 챕터별로 나눈 후 마크다운으로 변환
2. **Noise Removal**: 챕터 첫 페이지와 전체 페이지에서 노이즈 제거 (header/footer 패턴)
3. **Markdown Header Labeling**: 특정 패턴(Exercises, Key Terms 등)을 마크다운 헤더로 변환
4. **Header-based Chunking**: 마크다운 헤더를 기준으로 청크 분할
5. **Header-based Filtering**: 특정 헤더를 가진 청크 필터링
6. **Embedding with Clova**: Clova 임베딩 API를 사용한 벡터 생성 (RPM 고려)
7. **PostgreSQL pgvector Loading**: 임베딩 결과를 PostgreSQL pgvector에 적재

---
## Checkpoint 1: Chapter-based Markdown Conversion

PDF 파일에서 챕터 정보를 받아 각 챕터별로 페이지를 나누고 마크다운으로 변환합니다.

### 입력
- PDF 파일 경로
- 챕터별 시작 페이지 정보 (딕셔너리)

### 출력
- `chapter_markdowns`: 챕터별 마크다운 텍스트 (dict)

### 체크포인트 저장
- 변수: `checkpoint_1_chapter_markdowns`

### Step 1.1: 라이브러리 설치

필요한 라이브러리를 설치합니다.

In [ ]:
!pip install pymupdf4llm pymupdf -q

### Step 1.2: 라이브러리 임포트

필요한 라이브러리를 임포트합니다.

In [ ]:
import pymupdf4llm
import fitz  # PyMuPDF
import os

### Step 1.3: PDF 파일 및 챕터 정보 설정

처리할 PDF 파일 경로와 각 챕터의 시작 페이지 번호를 정의합니다.

In [ ]:
# PDF 파일 경로
pdf_path = "data/network.pdf"

# 챕터별 시작 페이지 번호 (PDF 페이지 번호)
chapter_start_pages = {
    "Chapter 1": 5,
    "Chapter 2": 47,
    "Chapter 3": 103,
    "Chapter 4": 179,
    "Chapter 5": 229,
    "Chapter 6": 287,
    "Chapter 7": 341,
    "Chapter 8": 373,
    "Chapter 9": 415,
}

# 챕터 정보 정렬 및 총 페이지 수 확인
sorted_chapters = sorted(chapter_start_pages.items(), key=lambda item: item[1])

doc = fitz.open(pdf_path)
total_pages = len(doc)
doc.close()

print(f"Target PDF: {pdf_path}")
print(f"Total Pages: {total_pages}")
print(f"Total Chapters: {len(sorted_chapters)}")
print(f"\nChapter Configuration:")
for chapter_name, start_page in sorted_chapters:
    print(f"  - {chapter_name}: Page {start_page}")

### Step 1.4: 챕터별 마크다운 변환 실행

각 챕터의 페이지 범위를 계산하고 pymupdf4llm을 사용하여 마크다운으로 변환합니다.

In [ ]:
chapter_markdowns = {}

print("=" * 60)
print("Starting Chapter-based Markdown Conversion")
print("=" * 60)

for i, (chapter_name, start_page) in enumerate(sorted_chapters):
    # 페이지 범위 계산 (0-indexed)
    start_idx = start_page - 1
    
    if i < len(sorted_chapters) - 1:
        # 다음 챕터 시작 전까지
        end_idx = sorted_chapters[i + 1][1] - 1
    else:
        # 마지막 챕터는 PDF 끝까지
        end_idx = total_pages
    
    # 페이지 범위 리스트 생성
    page_range = list(range(start_idx, end_idx))
    
    print(f"\n[{chapter_name}]")
    print(f"  Processing pages {start_idx + 1} to {end_idx} ({len(page_range)} pages)...")
    
    # pymupdf4llm을 사용하여 페이지별로 마크다운 변환
    chapter_md_text = pymupdf4llm.to_markdown(pdf_path, pages=page_range)
    
    # 챕터별 마크다운 저장
    chapter_markdowns[chapter_name] = chapter_md_text
    
    print(f"  ✓ Completed (Length: {len(chapter_md_text):,} characters)")

print("\n" + "=" * 60)
print(f"✓ All chapters converted successfully!")
print(f"  Total chapters: {len(chapter_markdowns)}")
print("=" * 60)

### Step 1.5: 체크포인트 저장 및 결과 확인

변환된 마크다운을 체크포인트로 저장하고 샘플 결과를 확인합니다.

In [ ]:
# 체크포인트 1 저장
checkpoint_1_chapter_markdowns = chapter_markdowns.copy()

print("✓ Checkpoint 1 saved successfully!")
print(f"  Variable: checkpoint_1_chapter_markdowns")
print(f"  Chapters: {list(checkpoint_1_chapter_markdowns.keys())}")
print(f"\n--- Sample: First 500 characters of Chapter 1 ---")
print(checkpoint_1_chapter_markdowns["Chapter 1"][:3000])
print("...")

---
## Checkpoint 2: Noise Removal

변환된 마크다운에서 불필요한 노이즈를 제거합니다.
- 전체 페이지: Header/Footer 영역의 페이지 번호, Copyright 문구 등 제거
- 챕터 첫 페이지: 추가로 중복되는 챕터 제목 등 제거

### 입력
- `checkpoint_1_chapter_markdowns`: 챕터별 원본 마크다운

### 출력
- `chapter_markdowns_cleaned`: 노이즈가 제거된 챕터별 마크다운 (dict)

### 체크포인트 저장
- 변수: `checkpoint_2_cleaned_markdowns`

### Step 2.1: 라이브러리 임포트

정규표현식 처리를 위한 라이브러리를 임포트합니다.

In [ ]:
import re

### Step 2.2: 노이즈 제거 패턴 설정

In [ ]:
# Header/Footer 확인 범위 설정 (각각 몇 줄씩 체크할지)
header_check_range = 5
footer_check_range = 5

# Header/Footer 무조건 삭제할 줄 수 (패턴 무관)
header_remove_lines = 0     # 상단 N줄을 무조건 삭제 (0 = 삭제 안함)
footer_remove_lines = 4     # 하단 N줄을 무조건 삭제 (0 = 삭제 안함)

# Header 패턴 (페이지 상단에서 제거할 패턴)
header_patterns = [
    r"Computer Networks: A Systems Approach, Release Version 6.1"
]

# Footer 패턴 (페이지 하단에서 제거할 패턴)
footer_patterns = []

# 챕터 첫 페이지 전용 패턴 (첫 페이지에서만 추가로 제거)
first_page_patterns = []

print("Noise Removal Configuration:")
print(f"  Header check range: {header_check_range} lines")
print(f"  Header unconditional removal: {header_remove_lines} lines")
print(f"  Footer check range: {footer_check_range} lines")
print(f"  Footer unconditional removal: {footer_remove_lines} lines")
print(f"  Header patterns: {len(header_patterns)} patterns")
print(f"  Footer patterns: {len(footer_patterns)} patterns")
print(f"  First page patterns: {len(first_page_patterns)} patterns")

### Step 2.3: 노이즈 제거 함수 정의

In [ ]:
def clean_page_markdown(md_text, is_first_page=False, show_removed=False, page_num=None):
    """
    페이지별 마크다운에서 노이즈를 제거합니다.
    
    Args:
        md_text (str): 원본 마크다운 텍스트
        is_first_page (bool): 챕터의 첫 페이지 여부
        show_removed (bool): 제거된 줄을 표시할지 여부
        page_num (int): 페이지 번호 (로깅용)
        
    Returns:
        str: 노이즈가 제거된 마크다운 텍스트
    """
    if not md_text:
        return ""
    
    lines = md_text.split('\n')
    lines_to_remove = set()
    removal_reasons = {}  # 제거 이유 저장
    
    # 1. 무조건 삭제할 줄 처리
    # Header: 상단 N줄 무조건 삭제
    if header_remove_lines > 0:
        for i in range(min(header_remove_lines, len(lines))):
            lines_to_remove.add(i)
            removal_reasons[i] = f"Header unconditional removal (top {header_remove_lines} lines)"
    
    # Footer: 하단 N줄 무조건 삭제
    if footer_remove_lines > 0:
        footer_start = max(0, len(lines) - footer_remove_lines)
        for i in range(footer_start, len(lines)):
            lines_to_remove.add(i)
            removal_reasons[i] = f"Footer unconditional removal (bottom {footer_remove_lines} lines)"
    
    # 2. Header 영역에서 패턴 매칭
    for i in range(min(header_check_range, len(lines))):
        if i in lines_to_remove:
            continue  # 이미 제거 대상이면 스킵
        
        line_stripped = lines[i].strip()
        
        # Header 패턴 체크
        for pattern in header_patterns:
            if re.search(pattern, line_stripped, re.IGNORECASE):
                lines_to_remove.add(i)
                removal_reasons[i] = f"Header pattern: {pattern}"
                break
    
    # 3. Footer 영역에서 패턴 매칭
    footer_start_idx = max(0, len(lines) - footer_check_range)
    for i in range(footer_start_idx, len(lines)):
        if i in lines_to_remove:
            continue  # 이미 제거 대상이면 스킵
        
        line_stripped = lines[i].strip()
        
        # Footer 패턴 체크
        for pattern in footer_patterns:
            if re.search(pattern, line_stripped, re.IGNORECASE):
                lines_to_remove.add(i)
                removal_reasons[i] = f"Footer pattern: {pattern}"
                break
    
    # 4. 챕터 첫 페이지인 경우 추가 패턴 체크
    if is_first_page:
        for i, line in enumerate(lines):
            if i in lines_to_remove:
                continue
            line_stripped = line.strip()
            for pattern in first_page_patterns:
                if re.search(pattern, line_stripped, re.IGNORECASE):
                    lines_to_remove.add(i)
                    removal_reasons[i] = f"First page pattern: {pattern}"
                    break
    
    # 제거된 줄 표시
    if show_removed and lines_to_remove:
        print(f"    [Page {page_num}] Removed {len(lines_to_remove)} line(s):")
        for i in sorted(lines_to_remove)[:10]:  # 처음 10개만 표시
            reason = removal_reasons.get(i, "Unknown")
            line_preview = lines[i][:60] + "..." if len(lines[i]) > 60 else lines[i]
            print(f"      Line {i}: '{line_preview}' ({reason})")
        if len(lines_to_remove) > 10:
            print(f"      ... and {len(lines_to_remove) - 10} more lines")
    
    # 제거 대상이 아닌 줄만 필터링
    cleaned_lines = [line for i, line in enumerate(lines) if i not in lines_to_remove]
    
    return '\n'.join(cleaned_lines)


print("✓ Noise removal function defined successfully!")
print("  - Header: Unconditional removal + pattern matching")
print("  - Footer: Unconditional removal + pattern matching")
print("  - First page: Pattern matching")

### Step 2.4: 챕터별 노이즈 제거 실행

Checkpoint 1에서 변환된 마크다운에 노이즈 제거 함수를 적용합니다.
각 챕터는 여러 페이지로 구성되어 있으므로, 페이지 단위로 분리하여 처리합니다.

In [ ]:
chapter_markdowns_cleaned = {}

print("=" * 60)
print("Starting Noise Removal")
print("=" * 60)

for i, (chapter_name, start_page) in enumerate(sorted_chapters):
    # 페이지 범위 계산 (0-indexed)
    start_idx = start_page - 1
    
    if i < len(sorted_chapters) - 1:
        end_idx = sorted_chapters[i + 1][1] - 1
    else:
        end_idx = total_pages
    
    print(f"\n[{chapter_name}]")
    print(f"  Processing pages {start_idx + 1} to {end_idx}...")
    
    # 페이지별로 변환하고 노이즈 제거 적용
    chapter_full_text = []
    
    for page_idx in range(start_idx, end_idx):
        # 첫 페이지 여부 확인
        is_first_page = (page_idx == start_idx)
        
        # 페이지를 마크다운으로 변환
        page_md = pymupdf4llm.to_markdown(pdf_path, pages=[page_idx])
        
        # 노이즈 제거 적용 (제거된 줄 표시)
        cleaned_md = clean_page_markdown(
            page_md, 
            is_first_page=is_first_page, 
            show_removed=True, 
            page_num=page_idx + 1
        )
        
        chapter_full_text.append(cleaned_md)
    
    # 챕터의 모든 페이지를 결합
    chapter_markdowns_cleaned[chapter_name] = "\n\n".join(chapter_full_text)
    
    original_length = len(checkpoint_1_chapter_markdowns.get(chapter_name, ""))
    cleaned_length = len(chapter_markdowns_cleaned[chapter_name])
    removed = original_length - cleaned_length
    
    print(f"  ✓ Completed")
    print(f"    Original: {original_length:,} chars")
    print(f"    Cleaned: {cleaned_length:,} chars")
    print(f"    Removed: {removed:,} chars")

print("\n" + "=" * 60)
print(f"✓ Noise removal completed for all chapters!")
print(f"  Total chapters: {len(chapter_markdowns_cleaned)}")
print("=" * 60)

### Step 2.5: 체크포인트 저장 및 결과 확인

노이즈가 제거된 마크다운을 체크포인트로 저장하고 샘플 결과를 확인합니다.

In [ ]:
# 체크포인트 2 저장
checkpoint_2_cleaned_markdowns = chapter_markdowns_cleaned.copy()

print("✓ Checkpoint 2 saved successfully!")
print(f"  Variable: checkpoint_2_cleaned_markdowns")
print(f"  Chapters: {list(checkpoint_2_cleaned_markdowns.keys())}")
print(f"\n--- Sample: First 500 characters of cleaned Chapter 1 ---")
print(checkpoint_2_cleaned_markdowns["Chapter 9"][:1000])
print("...")

---
## Checkpoint 3: Markdown Header Labeling

특정 키워드(Exercises, Attribution, Key Terms 등)를 찾아서 마크다운 헤더로 변환합니다.

### 입력
- `checkpoint_2_cleaned_markdowns`: 정제된 챕터별 마크다운
- `header_labels`: 변환할 키워드와 헤더 레벨 매핑 (dict)

### 출력
- `chapter_markdowns_labeled`: 헤더가 라벨링된 챕터별 마크다운 (dict)

### 체크포인트 저장
- 변수: `checkpoint_3_labeled_markdowns`

### Step 3.1: 헤더 라벨링 설정

특정 키워드를 마크다운 헤더로 변환하기 위한 설정을 정의합니다.

In [ ]:
# 헤더로 변환할 키워드와 헤더 레벨 매핑
# 키: 찾을 키워드, 값: 헤더 레벨 (2 = ##)
header_labels = {
    "Key Takeaway": 1
}

print("Header Labeling Configuration:")
print(f"  Total keywords: {len(header_labels)}")
print(f"\nKeywords to convert:")
for keyword, level in header_labels.items():
    header_symbol = "#" * level
    print(f"  - '{keyword}' → '{header_symbol} {keyword}'")

### Step 3.2: 헤더 라벨링 함수 정의

키워드를 찾아서 마크다운 헤더로 변환하는 함수를 정의합니다.

In [ ]:
import re

def label_headers(text, label_map, show_conversions=False):
    if not text:
        return ""
    
    lines = text.split('\n')
    new_lines = []
    conversions = []
    
    for line_num, line in enumerate(lines):
        line_stripped = line.strip()
        matched = False
        
        # 1. 마크다운 특수 기호들 제거 (앞뒤에 붙는 것들 위주)
        cleaned_line = line_stripped.strip("#* _~`>-")
        
        # 2. HTML 태그가 섞여 있을 경우 제거
        cleaned_line = re.sub(r'<[^>]*>', '', cleaned_line).strip()
        
        # 각 키워드에 대해 매칭 시도
        for keyword, level in label_map.items():
            if cleaned_line.lower() == keyword.lower():
                prefix = "#" * level
                new_line = f"{prefix} {keyword}"
                new_lines.append(new_line)
                matched = True
                
                if show_conversions:
                    conversions.append({
                        'line_num': line_num,
                        'original': line_stripped,
                        'converted': new_line,
                        'keyword': keyword
                    })
                break
        
        if not matched:
            new_lines.append(line)
    
    if show_conversions and conversions:
        print(f"    Found {len(conversions)} header(s) to label:")
        for conv in conversions:
            print(f"      Line {conv['line_num']}: '{conv['original']}' → '{conv['converted']}'")
    
    return '\n'.join(new_lines)

### Step 3.3: 챕터별 헤더 라벨링 실행

Checkpoint 2에서 정제된 마크다운에 헤더 라벨링을 적용합니다.

In [ ]:
chapter_markdowns_labeled = {}

print("=" * 60)
print("Starting Header Labeling")
print("=" * 60)

total_conversions = 0

for chapter_name, text in checkpoint_2_cleaned_markdowns.items():
    print(f"\n[{chapter_name}]")
    print(f"  Processing header labeling...")
    
    # 헤더 라벨링 적용 (변환 내용 표시)
    labeled_text = label_headers(text, header_labels, show_conversions=True)
    
    # 변환된 텍스트 저장
    chapter_markdowns_labeled[chapter_name] = labeled_text
    
    print(f"  ✓ Completed")

print("\n" + "=" * 60)
print(f"✓ Header labeling completed for all chapters!")
print(f"  Total chapters: {len(chapter_markdowns_labeled)}")
print("=" * 60)

### Step 3.4: 체크포인트 저장 및 결과 확인

헤더가 라벨링된 마크다운을 체크포인트로 저장하고 샘플 결과를 확인합니다.

In [ ]:
# 체크포인트 3 저장
checkpoint_3_labeled_markdowns = chapter_markdowns_labeled.copy()

print("✓ Checkpoint 3 saved successfully!")
print(f"  Variable: checkpoint_3_labeled_markdowns")
print(f"  Chapters: {list(checkpoint_3_labeled_markdowns.keys())}")

# 변환 예시 찾기
sample_chapter = None
for chapter_name, text in checkpoint_3_labeled_markdowns.items():
    if "## Key Terms" in text or "## Exercises" in text:
        sample_chapter = chapter_name
        break

if sample_chapter:
    print(f"\n--- Sample: {sample_chapter} (showing Key Terms/Exercises section) ---")
    text = checkpoint_3_labeled_markdowns[sample_chapter]
    
    lines = text.split('\n')
    for i, line in enumerate(lines):
        if line.startswith("## Key Terms") or line.startswith("## Exercises"):
            sample_text = '\n'.join(lines[i:min(i+10, len(lines))])
            print(sample_text)
            print("...")
            break
else:
    print("\n--- Sample: First 500 characters of Chapter 1 ---")
    print(checkpoint_3_labeled_markdowns["Chapter 1"][:500])
    print("...")